<a href="https://colab.research.google.com/github/KairaNithin/TPU-Repos/blob/main/TPU's_for_larger_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset

In [ ]:
mnli = load_dataset('glue', 'mnli')

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [ ]:
!pip install -U sentence-transformers
!pip install transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.7/dist-packages (1.2.0)


# ***With TPU*** 

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl
import os

os.environ['XLA_USE_BF16'] = "1"

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from torch.utils.data import DataLoader,RandomSampler



import random
import pickle
import re
import time
import datetime

import pandas as pd
import numpy as np


from sklearn.metrics import classification_report

from transformers import get_linear_schedule_with_warmup


import torch.nn as nn
from sklearn.metrics import f1_score


from tqdm import tqdm 
import gc
import os
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
from sklearn.metrics import f1_score

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("joeddav/xlm-roberta-large-xnli")


In [ ]:
class ourmodel(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = AutoModelForSequenceClassification.from_pretrained("joeddav/xlm-roberta-large-xnli")
  def forward(self,input_ids,attention_mask):
    outputs = self.model(input_ids,attention_mask)['logits']

    return outputs

In [ ]:

class create_dataset(torch.utils.data.Dataset):
  def __init__(self,dataset):
    self.dataset = dataset
  def __len__(self):
    return(len(self.dataset))
  def __getitem__(self,idx):
    self.inputs = self.dataset[idx]
    self.premise = self.inputs['premise']
    self.hypothesis = self.inputs['hypothesis']
    self.targets = self.inputs['label']

    self.inputs = tokenizer.encode_plus(
        self.premise,self.hypothesis,add_special_tokens=True,
        max_length = 128,
        pad_to_max_length=True,  # will be removed in future versions
        # padding=True,  # Not Working
        # padding = 'max_length',
        return_attention_mask=True
    )
    return({'input_ids':torch.tensor(self.inputs['input_ids']),
            'attention_mask':torch.tensor(self.inputs['attention_mask']),
            'targets':torch.tensor(self.targets)})

In [ ]:
mx = xmp.MpModelWrapper(ourmodel())

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.serialization as xser

import time
# model = ourmodel()
def map_fn(index, flags):
  ## Setup 

  torch.manual_seed(flags['seed'])

  device = xm.xla_device()  


  train_data = create_dataset(mnli['train'])
  valid_data = create_dataset(mnli['validation_matched'])


  train_sampler = torch.utils.data.distributed.DistributedSampler(
    train_data,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=True)
  
  test_sampler = torch.utils.data.distributed.DistributedSampler(
    valid_data,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=False)
  
  # Creates dataloaders, which load data in batches
  # Note: test loader is not shuffled or sampled
  train_loader = torch.utils.data.DataLoader(
      train_data,
      batch_size=flags['batch_size'],
      sampler=train_sampler,
      num_workers=flags['num_workers'],
      drop_last=True)

  test_loader = torch.utils.data.DataLoader(
      valid_data,
      batch_size=flags['batch_size'],
      sampler=test_sampler,
      shuffle=False,
      num_workers=flags['num_workers'],
      drop_last=True)
  

  model = mx.to(device).train()
  loss_fn = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters())
  # score = 0
  # prev_score = -1
  # patience = 0


  # ## Trains
  # train_start = time.time()
  # for epoch in range(flags['num_epochs']):
  #   xm.master_print("epoch----------------------------------------{}".format(epoch)) 
  #   if score > prev_score:
  #     xm.master_print("moving forward")
  #     open('model.pt',"wb").close()
  #     os.remove("model.pt")
  #     xser.save(model.state_dict(),'model.pt')
  #     patience = 0
  #   else :
  #     print("backtracking....")
  #     model.load_dict_state(torch.load('model.pt'))
  #     lr = lr/(2)
  #     optimizer = torch.optim.AdamW(model.parameters(),lr = lr)
  #     patience += 1
  #     if patience == 5:
  #       break
  #     if lr < 1e-8:
  #       break



    para_train_loader = pl.ParallelLoader(train_loader, [device]).per_device_loader(device)
    for batch_num, batch in enumerate(para_train_loader):
      xm.master_print("iter{}".format(batch_num))
      # data, targets = batch
      input_ids = batch['input_ids']
      attention_mask = batch['attention_mask']
      targets = batch['targets']

      # Acquires the network's best guesses at each class
      output = model(input_ids,attention_mask)

      # Computes loss
      loss = loss_fn(output, targets)

      # Updates model
      optimizer.zero_grad()
      loss.backward()

     
      xm.optimizer_step(optimizer)  # Note: barrier=True not needed when using ParallelLoader

  elapsed_train_time = time.time() - train_start
  print("Process", index, "finished training. Train time was:", elapsed_train_time) 


  # Evaluation
  # Sets net to eval and no grad context 
  model.eval()
  eval_start = time.time()
  with torch.no_grad():
    all_outputs = []
    all_targets = []
    flat_outputs = []
    flat_targets = []

    para_train_loader = pl.ParallelLoader(train_loader, [device]).per_device_loader(device)
    for batch_num, batch in enumerate(para_train_loader):
      input_ids = batch['input_ids']
      attention_mask = batch['attention_mask']
      targets = batch['targets']

      # Acquires the network's best guesses at each class
      output = model(input_ids,attention_mask)
      all_outputs.append(output)
    # all_outputs.append(outputs)
      all_targets.append(targets)
    flat_outputs = [torch.argmax(j.to('cpu')) for i in all_outputs for j in i]
    flat_targets = [j.to('cpu') for i in all_targets for j in i]
    # score = f1_score(flat_outputs,flat_targets)
    
  
  elapsed_eval_time = time.time() - eval_start
  print("Process", index, "finished evaluation. Evaluation time was:", elapsed_eval_time)
  print("f1_score = ",f1_score(flat_outputs,flat_targets))
  # print("Process", index, "guessed", num_correct, "of", total_guesses, "correctly for", num_correct/total_guesses * 100, "% accuracy.")

In [ ]:
def map_fn(index,flags):
  device = xm.xla_device()
  model = mx.to(device)
  xser.save(model.state_dict(),'model2')

In [ ]:
import os

In [ ]:
# Configures training (and evaluation) parameters
flags={}
flags['batch_size'] = 32
flags['num_workers'] = 2
flags['num_epochs'] = 5
flags['seed'] = 1234
import os

os.environ['XLA_USE_BF16'] = "1"

xmp.spawn(map_fn, args=(flags,), nprocs=8, start_method='fork')

KeyboardInterrupt: ignored